In [1]:
import pandas as pd

In [2]:
housing=pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
y_val=housing['medianHouseValue']

In [5]:
x_data=housing.drop('medianHouseValue',axis=1)

In [6]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_val, test_size=0.3, random_state=101)

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler=MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train=pd.DataFrame(data=scaler.transform(X_train),columns=X_train.columns,index=X_train.index)

In [14]:
X_test=pd.DataFrame(data=scaler.transform(X_test),columns=X_test.columns,index=X_test.index)

In [15]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [16]:
import tensorflow as tf

In [17]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [18]:
feat_cols=[age,rooms,bedrooms,pop,households,income]

In [19]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=100,shuffle=True)

In [20]:
model=tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\Satish\\AppData\\Local\\Temp\\tmpxfzil9n9', '_session_config': None, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_tf_random_seed': 1}


In [21]:
model.train(input_func,steps=100)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Satish\AppData\Local\Temp\tmpxfzil9n9\model.ckpt.
INFO:tensorflow:step = 1, loss = 7.40983e+11
INFO:tensorflow:Saving checkpoints for 100 into C:\Users\Satish\AppData\Local\Temp\tmpxfzil9n9\model.ckpt.
INFO:tensorflow:Loss for final step: 5.24067e+11.


In [22]:
predict_input_func=tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [23]:
pred_gen=model.predict(predict_input_func)

In [24]:
prediction=list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\Satish\AppData\Local\Temp\tmpxfzil9n9\model.ckpt-100


In [25]:
prediction

[{'predictions': array([ 957.24475098], dtype=float32)},
 {'predictions': array([ 1086.38305664], dtype=float32)},
 {'predictions': array([ 949.20202637], dtype=float32)},
 {'predictions': array([ 785.60046387], dtype=float32)},
 {'predictions': array([ 1017.67553711], dtype=float32)},
 {'predictions': array([ 903.08691406], dtype=float32)},
 {'predictions': array([ 992.61340332], dtype=float32)},
 {'predictions': array([ 895.71960449], dtype=float32)},
 {'predictions': array([ 887.26867676], dtype=float32)},
 {'predictions': array([ 708.28369141], dtype=float32)},
 {'predictions': array([ 906.90710449], dtype=float32)},
 {'predictions': array([ 1020.23431396], dtype=float32)},
 {'predictions': array([ 844.05249023], dtype=float32)},
 {'predictions': array([ 773.144104], dtype=float32)},
 {'predictions': array([ 1086.70080566], dtype=float32)},
 {'predictions': array([ 743.50970459], dtype=float32)},
 {'predictions': array([ 880.08782959], dtype=float32)},
 {'predictions': array([ 805.

In [33]:
final_preds=[]

for pred in prediction:
    final_preds.append(pred['predictions'])

In [34]:
from sklearn.metrics import mean_squared_error

In [35]:
mean_squared_error(y_test,final_preds) ** 0.5

237208.90202403886